### Importing and Encoding of the Train Dataset

### Machine Learning
### 

#### General Task:
"In this competition, you’re challenged to analyze a Google Merchandise Store (also known as GStore, where Google swag is sold) customer dataset to predict revenue per customer. Hopefully, the outcome will be more actionable operational changes and a better use of marketing budgets for those companies who choose to use data analysis on top of GA data." - Kaggle

#### Introduction and Issue of the paper:


#### Main Problem:


#### Mistakes:

We focused first just on the rows with a actual transaction revenue by overseeing that the rows with NAN values have the same importance. We thought there is no information about these samples and we dropped them. But after a while we figured that NAN stands for no transaction, so the amount of 0. 

#### Algorithms:
Just Light GBM was performing quite fast


#### Table of Contents:
1. Notebook: Cleaning, Wrangling, Encoding, Feature, Importance, Focus on the samples with an actual Revenue (Train)
2. Notbook: Cleaning, Wrangling, Encoding, Feature, Importance, Focus on the samples with an actual Revenue (Test)
2. Notbook: EDA, Data Visulaisation, Correlation, Analysis of NAN values
3. Notbook: Light GBM, Feature Importance, Hyperparamter Tuning, Fitting and Prediction
.... Further Trys, and Errors

In [ ]:
# imports
# I was trying a lot of different things during this notebook therefore there might be the one or two libaries which
# arn´t needed anymore


import requests                 # How Python gets the webpages
from bs4 import BeautifulSoup   # Creates structured, searchable object
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from pandas import ExcelWriter
from pandas import ExcelFile
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter 


import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
import gc

rcParams['figure.figsize'] = 30, 30
rcParams['font.size'] = 20
rcParams['axes.facecolor'] = 'white'

# plt.style.available
plt.style.use("seaborn-whitegrid")

#JSON flatten
import os
from pandas.io.json import json_normalize

from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier


## 1. Data Load & Json Flattening

Load the data and flattening out the JSON formates.
Otherwise you will have multi cells with different topics in one cell.

In [2]:
def load_df(csv_path='../datasets/train.csv', nrows = None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows = nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

print(os.listdir("../datasets"))

['googl_sample_submission_v2.csv', 'googl_test_v2.csv', 'googl_train_v2.csv', 'heavy_vehicles_2013-2017.xlsx', 'st1_sample_submission.csv', 'st1_test.csv', 'st1_train.csv', 'st_sample_submission.csv', 'st_test.csv', 'st_train.csv', 'test_adj_complete_featureselection.csv', 'test_identity.csv', 'test_transaction.csv', 'train_adj_complete_featureselection.csv', 'train_identity.csv', 'train_transaction.csv', 'yelp_reviews.csv']


We tried so many different ways to get the max data working on the laptop, but unfortunately neither solutions with chunking nor Dask was working. The best solution was to use this function above and the max number of rows was 600t.
So now lets load them in....

In [3]:
%%time
df_small = load_df('../datasets/googl_train_v2.csv', nrows = 600000)

Loaded googl_train_v2.csv. Shape: (600000, 60)
Wall time: 3min 22s


## 2. Data Cleaning
### 2.1 Small EDA and Cleaning of Columns refering to our target column

So after using the flatting function to get the hole features, we have 600t rows and 60 columns.

In [4]:
train = df_small
del df_small
train.shape

(600000, 60)

First we wanted to check our target column the transaction Revenue 

In [5]:
print('Total Lenght', len(train['totals.transactionRevenue']))
print('NA values', train['totals.transactionRevenue'].isna().sum())


Total Lenght 600000
NA values 593602


We see that its constists mainly out of nan Values, so in order to follow our target to reduce as much as we can we will drop the NAN values, because they might be useful, but we cant process so much data, so we focus on the actual transactions.

In [8]:
#Just that we can work with the data
train.rename(columns={'totals.transactionRevenue': 'total_transactionRevenue'}, inplace=True)

In [14]:
revenue = train['total_transactionRevenue']
revenue

0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
         ... 
599995    NaN
599996    NaN
599997    NaN
599998    NaN
599999    NaN
Name: total_transactionRevenue, Length: 600000, dtype: object

So we create a list with all the NAN Values and see, that its actual just NAN values inside.

In [18]:
delete1 = train[train.total_transactionRevenue.isnull()]
delete1.total_transactionRevenue.describe()

count       0
unique      0
top       NaN
freq      NaN
Name: total_transactionRevenue, dtype: object

Create a list with rows to drop with the columns names.......

In [21]:
liste = delete1.index

In [22]:
train.drop(liste, axis=0, inplace=True)

So our new sahpe is now with 6,3t rows, a bit more then 1% of the readed in dataset.

In [23]:
train.shape

(6398, 60)

### 2.2 Screening for cleaning of Features

In [24]:
# looking for empty values
missing= train.isna().sum()
percent = (train.isnull().sum()/train.isnull().count()*100).sort_values(ascending=False).round(2)
missing_data =pd.DataFrame([missing, percent], index = ['missing', 'percent'])
missing_percent=missing_data.transpose()
missing_percent.sort_values(by='percent', ascending=False)

,missing,percent
trafficSource.campaignCode,6398.0,100.00
totals.bounces,6398.0,100.00
trafficSource.adContent,6321.0,98.80
trafficSource.adwordsClickInfo.isVideoAd,6152.0,96.16
trafficSource.adwordsClickInfo.adNetworkType,6152.0,96.16
trafficSource.adwordsClickInfo.slot,6152.0,96.16
trafficSource.adwordsClickInfo.page,6152.0,96.16
trafficSource.adwordsClickInfo.gclId,6151.0,96.14
trafficSource.keyword,4952.0,77.40
totals.sessionQualityDim,4052.0,63.33


The table illustrates pretty good that some features subsist just out of NAN Values but the most of the features have 100 percent of the data. So lets kick the columns with more then 95 percent of the NAN values. 
We could let some of them in, but in order to reduce memory usage we will kick up to 95% of missing data...

In [25]:
delete = missing_percent[missing_percent.percent >= 95]
delete

,missing,percent
totals.bounces,6398.0,100.00
trafficSource.adContent,6321.0,98.80
trafficSource.adwordsClickInfo.page,6152.0,96.16
trafficSource.adwordsClickInfo.slot,6152.0,96.16
trafficSource.adwordsClickInfo.gclId,6151.0,96.14
trafficSource.adwordsClickInfo.adNetworkType,6152.0,96.16
trafficSource.adwordsClickInfo.isVideoAd,6152.0,96.16
trafficSource.campaignCode,6398.0,100.00


In [26]:
delete.index

Index(['totals.bounces', 'trafficSource.adContent',
       'trafficSource.adwordsClickInfo.page',
       'trafficSource.adwordsClickInfo.slot',
       'trafficSource.adwordsClickInfo.gclId',
       'trafficSource.adwordsClickInfo.adNetworkType',
       'trafficSource.adwordsClickInfo.isVideoAd',
       'trafficSource.campaignCode'],
      dtype='object')

In [29]:
train.drop(['totals.bounces', 'trafficSource.adContent',
       'trafficSource.adwordsClickInfo.page',
       'trafficSource.adwordsClickInfo.slot',
       'trafficSource.adwordsClickInfo.gclId',
       'trafficSource.adwordsClickInfo.adNetworkType',
       'trafficSource.adwordsClickInfo.isVideoAd',
       'trafficSource.campaignCode'],
       axis= 1, inplace = True)

Now we have 52 remaining fetaures

In [30]:
del delete
train.shape

(6398, 52)

## 3. Reducing Memory Usage
We will now change dtypes in order to reduce Memory Usage'

In [32]:
#source= https://wkirgsn.github.io/2018/02/10/auto-downsizing-dtypes/
from joblib import Parallel, delayed
import gc

#__AUTHOR__ = 'Kirgsn'

class Reducer:
    """
    Class that takes a dict of increasingly big numpy datatypes to transform
    the data of a pandas dataframe to in order to save memory usage.
    """
    memory_scale_factor = 1024**2  # memory in MB

    def __init__(self, conv_table=None):
        """
        :param conv_table: dict with np.dtypes-strings as keys
        """
        if conv_table is None:
            self.conversion_table = \
                {'int': [np.int8, np.int16, np.int32, np.int64],
                 'uint': [np.uint8, np.uint16, np.uint32, np.uint64],
                 'float': [np.float16, np.float32, ]}
        else:
            self.conversion_table = conv_table
            
        #gc.collect()

    def _type_candidates(self, k):
        for c in self.conversion_table[k]:
            i = np.iinfo(c) if 'int' in k else np.finfo(c)
            yield c, i

    def reduce(self, df, verbose=False):
        """Takes a dataframe and returns it with all data transformed to the
        smallest necessary types.

        :param df: pandas dataframe
        :param verbose: If True, outputs more information
        :return: pandas dataframe with reduced data types
        """
        ret_list = Parallel(n_jobs=-1, max_nbytes=None)(delayed(self._reduce)
                                                (df[c], c, verbose) for c in
                                                df.columns)

        return pd.concat(ret_list, axis=1)

    def _reduce(self, s, colname, verbose):

        # skip NaNs
        if s.isnull().any():
            if verbose:
                print(colname, 'has NaNs - Skip..')
            return s

        # detect kind of type
        coltype = s.dtype
        if np.issubdtype(coltype, np.integer):
            conv_key = 'int' if s.min() < 0 else 'uint'
        elif np.issubdtype(coltype, np.floating):
            conv_key = 'float'
        else:
            if verbose:
                print(colname, 'is', coltype, '- Skip..')
            print(colname, 'is', coltype, '- Skip..')
            return s

        # find right candidate
        for cand, cand_info in self._type_candidates(conv_key):
            if s.max() <= cand_info.max and s.min() >= cand_info.min:

                if verbose:
                    print('convert', colname, 'to', str(cand))
                return s.astype(cand)

        # reaching this code is bad. Probably there are inf, or other high numbs
        print(("WARNING: {} " 
               "doesn't fit the grid with \nmax: {} "
               "and \nmin: {}").format(colname, s.max(), s.min()))
        print('Dropping it..')

In [33]:
reducer = Reducer()
data_train = reducer.reduce(train)

In [35]:
train = data_train 
del data_train

- lets safe the formated df in csv. For this part our latop needed 4 hours. We didnt want to loose this time so safty first.

- Also important, we need this df later for our target to train the algorithms because in the next part we will encode the data but our traget should be excluded, so we safe it here.

In [36]:
train.to_csv('Google_reduced_train.csv')

And again, we search for missing values in features, now it should be better...

In [38]:
# looking for empty values
missing= train.isna().sum()
percent = (train.isnull().sum()/train.isnull().count()*100).sort_values(ascending=False).round(2)
missing_data =pd.DataFrame([missing, percent], index = ['missing', 'percent'])
missing_percent=missing_data.transpose()
missing_percent.sort_values(by='percent', ascending=False).head()

,missing,percent
trafficSource.keyword,4952.0,77.40
totals.sessionQualityDim,4052.0,63.33
totals.newVisits,3795.0,59.32
trafficSource.referralPath,3395.0,53.06
trafficSource.isTrueDirect,2514.0,39.29


## 4. Encoding the DF for our algorithms
#### 4.1 Encoding
Encode strings, objects and floats

In [2]:
train = pd.read_csv('Google_reduced_train.csv')


In [3]:
train.drop('Unnamed: 0', axis = 1, inplace=True)

Lets check our data shape

In [4]:
train.shape

(6398, 52)

Unfortunately we have to drop this two columns because they were not flattend right and we have no remaining time to deal with this problem...

In [5]:
train.drop('customDimensions', axis=1, inplace = True)

In [6]:
train.drop('hits', axis=1, inplace = True)

In [7]:
train.shape

(6398, 50)

We safe here our original train df, because while the encoding also the column labels get changed so we can easily get them back later

In [8]:
# here we are safing our dataset, because label Encoder was changing our train dataset
original = train.copy()

In [9]:
train_RM = train

Random Forest was complaining about float formats, so we use the encoder to perform RM

This error is necessary in order to run our imputer, we made many different tries but that was the only way it was working ;) so we let the error happen ...

In [10]:
for f in train_RM.columns:
   # if f != 'total_transactionRevenue':
        if train_RM[f].dtype=='float': 
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(train_RM[f].values))
            train_RM[f] = lbl.transform(list(train_RM[f].values))

ValueError: y contains previously unseen labels: [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 

In [11]:
for f in train_RM.columns:
   # if f != 'total_transactionRevenue':
        if train_RM[f].dtype=='str': 
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(train_RM[f].values))
            train_RM[f] = lbl.transform(list(train_RM[f].values))

In [12]:
for f in train_RM.columns:
    #if f != 'total_transactionRevenue':
        if train_RM[f].dtype=='object': 
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(train_RM[f].values))
            train_RM[f] = lbl.transform(list(train_RM[f].values))

ValueError: y contains previously unseen labels: [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 

### 4.2 Dealing with NAN Values
Now we want to deal smart with the remaining NAN values with the Imputer and Bayesian Ridge.

In [13]:
# idea f rom thissource: https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

from sklearn.impute import SimpleImputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor

 # estimators: BayesianRidge(), DecisionTreeRegressor(max_features='sqrt', random_state=0), 
 # ExtraTreesRegressor(n_estimators=10, random_state=0),
 # KNeighborsRegressor(n_neighbors=15)

imp = IterativeImputer(estimator = BayesianRidge(), 
                       missing_values=np.nan, 
                       sample_posterior=False, 
                       max_iter=10, 
                       tol=0.001, 
                       n_nearest_features=10, # amount of features,
                       imputation_order = 'ascending',
                       min_value = None,
                       max_value = None, 
                       verbose = 2, # controls the debug messages 
                       random_state = None,
                       add_indicator = True,
                       initial_strategy='mean')
imp.fit(train_RM)
imputed_train = pd.DataFrame(data=imp.transform(train_RM),
                             dtype='int')

[IterativeImputer] Completing matrix with shape (6398, 50)
[IterativeImputer] Ending imputation round 1/10, elapsed time 0.21
[IterativeImputer] Early stopping criterion reached.
[IterativeImputer] Completing matrix with shape (6398, 50)
[IterativeImputer] Ending imputation round 1/1, elapsed time 0.00


In [14]:
train_RM = imputed_train

Now run them again and look if they work:

In [15]:
for f in train_RM.columns:
    #if f != 'total_transactionRevenue':
        if train_RM[f].dtype=='object': 
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(train_RM[f].values))
            train_RM[f] = lbl.transform(list(train_RM[f].values))

In [16]:
for f in train_RM.columns:
    #if f != 'total_transactionRevenue':
        if train_RM[f].dtype=='float': 
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(train_RM[f].values))
            train_RM[f] = lbl.transform(list(train_RM[f].values))

Like we said above, the encoding algorithm had also encoded the columns, so we have to bring them back in the normal format:

In [17]:
train_RM.head(1)

,0,1,2,3,4,5,6,7,8,9,...,44,45,46,47,48,49,50,51,52,53
0,3,20171016,-2147483648,0,1508200705,4,1508200705,1,0,0,...,12,4,2,0,37,1,1,0,0,0


Also the encoder created avery time a new column, we have to get rid of the last 4 columns..

In [18]:
train.shape

(6398, 50)

In [19]:
train_RM.shape

(6398, 54)

In [20]:
train_RM.drop(50, axis=1, inplace = True)

In [21]:
train_RM.drop(51, axis=1, inplace = True)
train_RM.drop(52, axis=1, inplace = True)
train_RM.drop(53, axis=1, inplace = True)

Now get the names again on the df

In [22]:
namesList = train.columns

In [23]:
train_RM.columns = namesList

In [24]:
#changing it back to the original version and drop original due to memory usage reduction
train = original

del original

In [25]:
train_RM

,channelGrouping,date,fullVisitorId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.browserVersion,device.browserSize,...,totals.transactions,total_transactionRevenue,totals.totalTransactionRevenue,trafficSource.campaign,trafficSource.source,trafficSource.medium,trafficSource.keyword,trafficSource.adwordsClickInfo.criteriaParameters,trafficSource.referralPath,trafficSource.isTrueDirect
0,3,20171016,-2147483648,0,1508200705,4,1508200705,1,0,0,...,1,15190000,17190000,1,12,4,2,0,37,1
1,5,20171016,-2147483648,0,1508192636,11,1508192636,1,0,0,...,1,8000000,13000000,1,0,0,20,0,0,1
2,3,20171016,-2147483648,0,1508162218,6,1508162218,6,0,0,...,1,57300000,65300000,1,12,4,2,0,37,1
3,5,20171016,-2147483648,0,1508189401,17,1508189401,1,0,0,...,1,18230000,25230000,1,0,0,20,0,0,1
4,5,20171016,-2147483648,0,1508190484,1,1508190484,1,0,0,...,1,12310000,16310000,1,0,0,20,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6393,3,20171023,-2147483648,0,1508786788,2,1508786788,1,0,0,...,1,135710000,143710000,1,12,4,2,0,37,1
6394,5,20171023,-2147483648,0,1508774504,3,1508774504,1,0,0,...,1,53850000,59850000,1,0,0,20,0,0,1
6395,5,20171023,-2147483648,0,1508771038,1,1508771038,1,0,0,...,2,136800000,180170000,1,0,0,20,0,0,1
6396,5,20171023,-2147483648,0,1508788879,2,1508788879,1,0,0,...,1,30520000,39520000,1,0,0,20,0,0,1


In [27]:
train_RM.to_csv('Google_Encoded_data.csv')

## 5. Feature Selection via Random Forest
Now we can apply our Classification Algorithm Random Forest to get Features they are unnecessary. 
We therefore want to filter the features with no value for the Random Forest.
##### The Random Forest Score is builded this way:

By training a tree, it can be computed how much each feature decreases the weighted impurity in a tree. For a forest, the impurity decrease from each feature can be averaged and the features are ranked according to this measure.

In [149]:
clf = RandomForestClassifier()
x = train_RM.drop('total_transactionRevenue', axis=1)
X = x
y = train.total_transactionRevenue

#feature selection
#When training a tree, it can be computed how much each feature decreases the weighted impurity in a tree. 
#For a forest, the impurity decrease from each feature can be averaged and the features are ranked according to this measure.

clf.fit(X,y)

rank = pd.DataFrame(clf.feature_importances_,index= X.columns)
rank.columns = ['Random Forest Score']
rank.sort_values(by = 'Random Forest Score' , ascending = False)

C:\Users\Tobias\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


,Random Forest Score
totals.totalTransactionRevenue,0.115605
totals.timeOnSite,0.085248
visitId,0.084487
visitStartTime,0.082232
date,0.076461
totals.hits,0.076142
totals.sessionQualityDim,0.074208
totals.pageviews,0.073890
visitNumber,0.050558
geoNetwork.networkDomain,0.044261


So here is the list of the random forest. We see that about half of the features got no value.
Like explained above, features with value 0 had no impact on impurity decrease and are therefore useless for the RM.
Maybe we will kick some of the features in the next Notebooks for better efficiency.